In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import chime
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
%load_ext chime

# Model selection for the reservation data set
Preparing for Kaggle competion - the Playground Series - Season 3, Episode 7

https://www.kaggle.com/competitions/playground-series-s3e7
## Individual classifiers

### Note
I use the mean_test_score of cross validation in this notebook to compare model performance with different hyperparameters and see if the difference is big enough to continue optimization

# Data preparation

In [3]:
train = pd.read_csv('./data/train__dataset.csv'); train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,4,0,0,0,118,2017,12,28,1,0,0,0,110.80,2,0
1,2,1,0,2,0,0,0,17,2018,4,14,1,0,0,0,145.00,0,1
2,1,0,1,5,0,0,0,349,2018,10,4,0,0,0,0,96.67,0,1
3,1,0,2,4,0,0,0,69,2018,6,12,0,0,0,0,120.00,0,1
4,2,0,0,4,1,0,0,11,2018,1,20,1,0,0,0,69.50,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18132,1,0,0,2,0,0,0,103,2018,4,19,0,0,0,0,115.00,0,1
18133,2,0,0,3,0,0,0,129,2018,8,10,1,0,0,0,88.01,1,0
18134,2,0,0,1,0,0,0,90,2018,7,13,1,0,0,0,105.30,0,1
18135,2,0,0,3,0,0,0,18,2018,11,10,1,1,0,1,123.33,1,0


The dataset is small so I will not split a validation set, rather will base parameter selection on cross-validation results.

In [4]:
X_train = train.drop(columns='booking_status')
y_train = train.booking_status

In [5]:
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,0.296940,-0.262944,0.216265,1.255698,-0.506205,-0.180783,-0.435754,0.376664,-2.139435,1.484357,1.406570,0.300026,-0.160413,-0.060638,-0.088329,0.206385,1.754593
1,0.296940,2.182701,-0.928626,-0.146506,-0.506205,-0.180783,-0.435754,-0.789492,0.467413,-1.115651,-0.189318,0.300026,-0.160413,-0.060638,-0.088329,1.170495,-0.783738
2,-1.641023,-0.262944,0.216265,1.956800,-0.506205,-0.180783,-0.435754,3.043812,0.467413,0.834355,-1.329238,-1.248072,-0.160413,-0.060638,-0.088329,-0.191944,-0.783738
3,-1.641023,-0.262944,1.361156,1.255698,-0.506205,-0.180783,-0.435754,-0.189095,0.467413,-0.465649,-0.417302,-1.248072,-0.160413,-0.060638,-0.088329,0.465737,-0.783738
4,0.296940,-0.262944,-0.928626,1.255698,1.083310,-0.180783,-0.435754,-0.858769,0.467413,-2.090654,0.494634,0.300026,-0.160413,-0.060638,-0.088329,-0.957876,0.485428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18132,-1.641023,-0.262944,-0.928626,-0.146506,-0.506205,-0.180783,-0.435754,0.203472,0.467413,-1.115651,0.380642,-1.248072,-0.160413,-0.060638,-0.088329,0.324785,-0.783738
18133,0.296940,-0.262944,-0.928626,0.554596,-0.506205,-0.180783,-0.435754,0.503671,0.467413,0.184353,-0.645286,0.300026,-0.160413,-0.060638,-0.088329,-0.436073,0.485428
18134,0.296940,-0.262944,-0.928626,-0.847608,-0.506205,-0.180783,-0.435754,0.053373,0.467413,-0.140648,-0.303310,0.300026,-0.160413,-0.060638,-0.088329,0.051338,-0.783738
18135,0.296940,-0.262944,-0.928626,0.554596,-0.506205,-0.180783,-0.435754,-0.777946,0.467413,1.159356,-0.645286,0.300026,6.233902,-0.060638,0.495072,0.559610,0.485428


# Best models
Assembling data from tuning individual classifiers

### Random forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
best_params_rf =\
{'max_depth': 100,
 'max_features': 'sqrt',
 'max_samples': 1.0,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [8]:
random_forest = RandomForestClassifier(n_jobs=-1,
                                       random_state=8,
                                       bootstrap=True,
                                      **best_params_rf)

In [9]:
best_score_rf = 0.9495807215415704

### XGBoost

In [10]:
from xgboost import XGBClassifier

In [11]:
best_params_xgb =\
{'learning_rate': 0.2,
 'max_depth': 8,
 'n_estimators': 120,
 'reg_alpha': 0.05,
 'reg_lambda': 0.3}

In [12]:
xgboost_clf = XGBClassifier(random_state=8,
                         max_leaves=0,
                         n_jobs=-1,
                         **best_params_xgb)

In [13]:
best_score_xgb = 0.9498993319202901

### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
best_params_knn = {'n_neighbors': 40, 'p': 1, 'weights': 'distance'}

In [16]:
knn = KNeighborsClassifier(algorithm='auto',
                           metric='minkowski',
                           n_jobs=-1,
                          **best_params_knn)

In [17]:
best_score_knn = 0.9294982428519282

### AdaBoost

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [19]:
best_params_ada =\
{'learning_rate': 0.005,
 'n_estimators': 150}

In [20]:
best_params_base =\
{'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_split': 40}

In [21]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=8,
                                                              **best_params_base),
                         random_state=8,
                        **best_params_ada)

In [22]:
best_score_ada = 0.9477394590502275

# Ensembles
## Voting classifier (unweighted)

In [23]:
from sklearn.ensemble import VotingClassifier

In [24]:
vote_with_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)

In [25]:
%%chime
scores_with_knn = cross_val_score(vote_with_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


In [27]:
scores_with_knn.mean()

0.9514909982200948

In [28]:
scores_with_knn.std()

0.0022603393023085195

In [29]:
vote_without_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    voting='soft'
)

In [30]:
%%chime
scores_without_knn = cross_val_score(vote_without_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


In [31]:
scores_without_knn.mean()

0.952538221161847

In [32]:
scores_without_knn.std()

0.0019765545278492597

## Voting classifier (weighted)

I use best scores of each classifier as its weight

In [33]:
weights_with_knn = [best_score_rf, best_score_xgb, best_score_knn, best_score_ada]

In [34]:
weighted_vote_with_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft',
    weights=weights_with_knn
)

In [35]:
%%chime
weighted_scores_with_knn = cross_val_score(weighted_vote_with_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


In [36]:
weighted_scores_with_knn.mean()

0.9515343332058656

In [37]:
weighted_scores_with_knn.std()

0.0022599977905760037

In [38]:
weights_without_knn = [best_score_rf, best_score_xgb, best_score_ada]

In [39]:
weighted_vote_without_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    voting='soft',
    weights=weights_without_knn
)

In [40]:
%%chime
weighted_scores_without_knn = cross_val_score(weighted_vote_without_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


In [41]:
weighted_scores_without_knn.mean()

0.9525386354235073

In [42]:
weighted_scores_without_knn.std()

0.001975721557816767

### Conclusion on voting
- Voting improves results slightly compared to individual classifiers
- It is better to use top classifiers without KNN. For top classifiers, using their scores as weights does not result in significant improvement

## Stacking  classifiers

In [43]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

### Final estimator: logistic regression
With KNN

In [44]:
stack_1 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    verbose=3
)

In [45]:
%%chime
scores_1 = cross_val_score(stack_1,
                           X_train, y_train,
                           scoring='roc_auc',
                           cv=5)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   17.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

In [47]:
scores_1.mean()

0.9521699145065291

In [50]:
scores_1.std()

0.00252548771723985

### Final estimator: logistic regression
Without KNN

In [51]:
stack_2 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    verbose=3
)

In [52]:
%%chime
scores_2 = cross_val_score(stack_2,
                           X_train, y_train,
                           scoring='roc_auc',
                           cv=5)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   17.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

In [53]:
scores_2.mean()

0.9521038208466376

In [54]:
scores_2.std()

0.0022942235486286453

### Final estimator: XGBoost
Without KNN

In [23]:
from sklearn.ensemble import StackingClassifier

In [24]:
stack_3 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

params = {
    'final_estimator__n_estimators': [25, 50, 100],
    'final_estimator__max_depth': [2, 4, 6],
    'final_estimator__learning_rate': [0.01, 0.1, 0.3],
    'final_estimator__reg_alpha': [0, 0.1],
    'final_estimator__reg_lambda': [0, 0.1]
}

In [41]:
params = {
    'final_estimator__n_estimators': [70, 100, 150, 200],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.05, 0.1, 0.2],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [42]:
model_3 = GridSearchCV(estimator=stack_3,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [43]:
%%time
%%chime
model_3.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  33.7s
[CV 2/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  33.1s
[CV 3/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  33.0s
[CV 4/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.954 total time=  33.5s
[CV 5/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimato

[CV 5/5] END final_estimator__learning_rate=0.05, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  33.4s
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  33.1s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.956 total time=  33.2s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  33.2s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimat

[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.953 total time=  33.7s
[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.949 total time=  33.2s
[CV 1/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  34.7s
[CV 2/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  33.3s
[CV 3/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimat

[CV 3/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.948 total time=  33.6s
[CV 4/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.950 total time=  33.7s
[CV 5/5] END final_estimator__learning_rate=0.2, final_estimator__max_depth=3, final_estimator__n_estimators=200, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.947 total time=  33.3s
CPU times: user 46min 28s, sys: 1min 43s, total: 48min 12s
Wall time: 1h 7min 26s


In [44]:
model_3.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 70,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [45]:
model_3.best_score_

0.9521756314451568

In [46]:
pd.Series(model_3.cv_results_['mean_test_score']).describe()

count    24.000000
mean      0.951557
std       0.000716
min       0.949249
25%       0.951262
50%       0.951865
75%       0.952046
max       0.952176
dtype: float64

### With KNN

In [49]:
stack_4 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

params = {
    'final_estimator__n_estimators': [50, 70, 100],
    'final_estimator__max_depth': [2, 4],
    'final_estimator__learning_rate': [0.05, 0.1, 0.2],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [58]:
params = {
    'final_estimator__n_estimators': [80, 100, 120, 150, 200],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.1],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [59]:
model_4 = GridSearchCV(estimator=stack_4,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [60]:
%%time
%%chime
model_4.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.4s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.956 total time=  37.2s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.1s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  37.1s
[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_e

[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=120, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.1s
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.3s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  37.2s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.5s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_esti

In [61]:
model_4.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 80,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [62]:
model_4.best_score_

0.9531706708568304

In [63]:
pd.Series(model_4.cv_results_['mean_test_score']).describe()

count    10.000000
mean      0.952920
std       0.000258
min       0.952283
25%       0.952868
50%       0.952964
75%       0.953087
max       0.953171
dtype: float64

### Results so far
The best result is achieved when KNN is included in a stacked classifier with XGBoost as the final estimator, although it is computationally expensive. I will try to add logistic regression as well

### With KNN and logistic regression

In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
best_params_lr = {'C': 0.1, 'max_iter': 50, 'penalty': 'l2'}
lr = LogisticRegression(class_weight='balanced',
                        random_state=8,
                        solver='liblinear',
                        **best_params_lr)

In [67]:
stack_5 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada),
        ('lr', lr)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

In [68]:
params = {
    'final_estimator__n_estimators': [50, 80, 100],
    'final_estimator__max_depth': [2, 4],
    'final_estimator__learning_rate': [0.01, 0.1, 0.5],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [69]:
model_5 = GridSearchCV(estimator=stack_5,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [70]:
%%time
%%chime
model_5.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.947 total time=  39.0s
[CV 2/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.6s
[CV 3/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.947 total time=  37.3s
[CV 4/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.949 total time=  37.5s
[CV 5/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator

[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.5s
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.4s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.956 total time=  37.4s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.5s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estim

[CV 4/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.6s
[CV 5/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.946 total time=  37.4s
[CV 1/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.944 total time=  37.5s
[CV 2/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.949 total time=  38.0s
[CV 3/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator

In [71]:
model_5.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 80,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [72]:
model_5.best_score_

0.9531468487214768

In [73]:
pd.Series(model_5.cv_results_['mean_test_score']).describe()

count    18.000000
mean      0.950803
std       0.002509
min       0.945295
25%       0.949192
50%       0.951922
75%       0.952916
max       0.953147
dtype: float64

Addition of logistic regression did not improve results

# Conclusion
The best result is achieved when KNN is included in a stacked classifier with XGBoost as the final estimator, although it is computationally expensive.